In [ ]:
from post_processing import *
from const import ROOT
from const import *
from post_processing import train_file_list, test_file_list
import pickle
def rle_decode(rle_mask,size):
    '''
    rle_mask: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = rle_mask.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(size*size, dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(size,size)

In [ ]:
mask = rle_decode("1 605 607 99 708 99 809 98 910 97 1011 97 1112 96 1213 95 1314 95 1415 94 1516 94 1617 94 1718 93 1819 93 1920 93 2021 93 2122 93 2223 94 2324 93 2425 93 2526 92 2627 92 2728 91 2829 90 2930 89 3031 89 3132 90 3233 91 3334 93 3435 95 3536 96 3637 95 3738 95 3839 94 3940 92 4041 91 4142 90 4243 89 4344 88 4445 87 4546 87 4647 86 4748 85 4849 85 4950 84 5051 83 5152 82 5253 82 5354 81 5455 81 5556 80 5657 80 5758 80 5859 79 5960 78 6061 78 6162 77 6263 77 6364 77 6465 77 6566 78 6667 78 6768 79 6869 79 6970 79 7071 79 7172 79 7273 80 7374 80 7475 81 7576 81 7677 81 7778 81 7879 81 7980 81 8081 81 8182 81 8283 81 8384 81 8485 81 8586 81 8687 81 8788 81 8889 82 8990 82 9091 82 9192 82 9293 82 9394 83 9495 83 9596 84 9697 86 9798 88 9899 91 10000 93 10101 94",101)
Image.fromarray(mask*255)

In [ ]:
height, width = 101, 101

if height % 32 == 0:
    y_min_pad = 0
    y_max_pad = 0
else:
    y_pad = 32 - height % 32
    y_min_pad = int(y_pad / 2)
    y_max_pad = y_pad - y_min_pad

if width % 32 == 0:
    x_min_pad = 0
    x_max_pad = 0
else:
    x_pad = 32 - width % 32
    x_min_pad = int(x_pad / 2)
    x_max_pad = x_pad - x_min_pad
print((y_min_pad,y_max_pad, x_min_pad, x_max_pad))

In [ ]:
arr1 = pickle.load( open( "ocnet_all_sigmoids.p", "rb" ) )
arr2 = pickle.load( open( "resnet34_all_sigmoids.p", "rb" ) )
import cv2

arr2 = arr2[:, y_min_pad:128 - y_max_pad, x_min_pad:128 - x_max_pad]

arr1 = arr1[:, y_min_pad:128 - y_max_pad, x_min_pad:128 - x_max_pad]

arr = (arr1+arr2)/2
#arr= arr>0.5
dicts={}

In [ ]:
print(arr1.mean())
print(arr2.mean())
print(arr.mean())

In [ ]:
arr3 = arr>0.38
dicts={}
arr1.shape
mean = 0.5*np.ones((101,101))

In [ ]:
coverage=[]
for i in tqdm(range(arr3.shape[0])):
    mask=arr3[i,:,:]
    coverage.append(arr3[i,:,:].sum()/(101*101))
    dicts[i]=np.array((mask-mean)**2).mean()
    #im = Image.fromarray(mask*255)
    #im.show()
    #break
dicts=np.array(dicts)

In [ ]:
import matplotlib.pyplot as plt
plt.hist(coverage, bins=50)  # arguments are passed to np.histogram
plt.title("higher means better")
plt.show()

In [ ]:
ct=0
for i in coverage:
    if i ==0: ct=ct+1
ct

In [ ]:
df2 = pd.read_csv("256_arith_mean.csv")
ct=0
for i in tqdm(range(len(test_file_list))):
    #print(row[1]["rle_mask"] is float)
    try:
        s=0
        if coverage[i]==0:
            #print("s")
            s=s+1
        #print(s)
        if s>0: 
            ct=ct+1
            df2.loc[df2['id'] == df2.iloc[i]['id'],'rle_mask']=' '.join(map(str, ""))        
    except ValueError:
        a=1
print("setting {} masks to empty".format(ct))

ct=0
for i in tqdm(range(len(df2))):
    #print(row[1]["rle_mask"] is float)
    try:
        s=0
        if type(df2.iloc[i]["rle_mask"]) is float or df2.iloc[i]["rle_mask"] == '' :
            #print("s")
            s=s+1
        #print(s)
        if s>0: 
            ct=ct+1
            #df2.loc[df2['id'] == df.iloc[i]['id'],'rle_mask']=' '.join(map(str, ""))        
    except ValueError:
        a=1
print("{} empty masks in total".format(ct))

In [ ]:
df2.to_csv("256_60models_new038.csv",index=False)

In [ ]:
df2 = pd.read_csv("256_60models_new038.csv")
ct=0
for i in tqdm(range(len(test_file_list))):
    #print(row[1]["rle_mask"] is float)
    try:
        s=0
        if coverage[i]==0:
            #print("s")
            s=s+1
        #print(s)
        if s>0: 
            ct=ct+1
            df2.loc[df2['id'] == df2.iloc[i]['id'],'rle_mask']=' '.join(map(str, "1 1"))
        else:
            df2.loc[df2['id'] == df2.iloc[i]['id'],'rle_mask']=' '.join(map(str, ""))
    except ValueError:
        a=1
print("setting {} masks to empty".format(ct))

In [ ]:
df2.to_csv("256_50models_038.csv",index=False)

In [ ]:
df2 = pd.read_csv("256_arith_mean_smoothed_only.csv") #7331
#df2 = pd.read_csv("0875_baseline.csv") #7617
ct=0
for i in tqdm(range(len(df2))):
    #print(row[1]["rle_mask"] is float)
    try:
        s=0
        if type(df2.iloc[i]["rle_mask"]) is float or df2.iloc[i]["rle_mask"] == '' :
            #print("s")
            s=s+1
        #print(s)
        if s>0: 
            ct=ct+1
            #df2.loc[df2['id'] == df.iloc[i]['id'],'rle_mask']=' '.join(map(str, ""))        
    except ValueError:
        a=1
print("{} empty masks in total".format(ct))

In [ ]:
strong={}
a=dicts.mean()+2*dicts.std()
print(a)
for i in tqdm(range(arr.shape[0])):
    if dicts[i]<a and arr[i,:,:].sum()<0.9 and arr[i,:,:].std()>0.1:
        strong[i]=dicts[i]
print("num of strong examples ",len(weak))
#max(weak.values())
for name, age in strong.items():    # for name, age in dictionary.iteritems():  (for Python 2.x)
    if age == min(strong.values()):
        print(name)
pickle.dump(list(weak.keys()),open("256_strong_samples.p","wb"))

In [ ]:
i=9821
for j in tqdm(range(arr.shape[0])):
    mask=arr[i,:,:]
    #print(mask)
    #dicts.append(np.array((mask-mean)**2).mean())
    im = Image.fromarray(mask*255)
    im.show()
    
    mask2=arr2[i,:,:]
    #print(mask)
    #dicts.append(np.array((mask-mean)**2).mean())
    im2 = Image.fromarray(mask2*255)
    im2.show()
    
    mask3=arr1[i,:,:]
    #print(mask)
    #dicts.append(np.array((mask-mean)**2).mean())
    im3 = Image.fromarray(mask3*255)
    im3.show()
    break

In [ ]:
df = pd.read_csv("256_45models.csv")

In [ ]:
for i in range(0,2015):
    row = df.iloc[i]
    #print(row['rle_mask'])
    mask=rle_decode(row['rle_mask'],size=101)
    im = Image.fromarray(mask*255)
    im.show()

In [ ]:
test_path = os.path.join(ROOT, 'test')
test_file_list = glob.glob(os.path.join(test_path, 'images', '*.png'))
test_file_list = [f.split('/')[-1].split('.')[0] for f in test_file_list]
train_path = os.path.join(ROOT, 'train')
train_file_list = glob.glob(os.path.join(train_path, 'images', '*.png'))
train_file_list = [f.split('/')[-1].split('.')[0] for f in train_file_list]

In [ ]:
indx=500
img = Image.open(ROOT+'/test/images/'+test_file_list[indx]+'.png')

In [ ]:
depths = pd.read_csv(ROOT+"/depths.csv")

In [ ]:
from PIL import ImageStat
import math
stat = ImageStat.Stat(img)

In [ ]:
gs = (math.sqrt(0.241*(r**2) + 0.691*(g**2) + 0.068*(b**2)) for r,g,b in img.getdata())
sum(gs)/stat.count[0]

In [ ]:
depths.loc[depths['id'] == test_file_list[indx]]['z'].values[0]
brightness=[]
depth=[]
coverage=[]

In [ ]:
for i in tqdm(range(len(train_file_list))):
    img = Image.open(ROOT+'/train/images/'+train_file_list[i]+'.png')
    mask = np.array(Image.open(ROOT+'/train/masks/'+train_file_list[i]+'.png'))
    coverage.append(mask.sum()/101*101)
    stat = ImageStat.Stat(img)
    gs = (math.sqrt(0.241*(r**2) + 0.691*(g**2) + 0.068*(b**2)) for r,g,b in img.getdata())
    brightness.append(sum(gs)/stat.count[0])
    depth.append(depths.loc[depths['id'] == train_file_list[i]]['z'].values[0])

In [ ]:
depths=np.array([depth]).reshape(-1,1)
coverages=np.array([coverage]).reshape(-1,1)
brightnesses=np.array([brightness]).reshape(-1,1)

In [ ]:
depths

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

n_iters = 5
preds_buf = []
err_buf = []
for i in range(n_iters): 
    x_train, x_valid, y_train, y_valid = train_test_split(depths,coverages, test_size=0.10, random_state=i)
    
    x_train=scaler.fit_transform(x_train)
    y_train=scaler.fit_transform(y_train)
    
    x_valid=scaler.fit_transform(x_valid)
    y_valid=scaler.fit_transform(y_valid)
    lgb_train = lgb.Dataset(x_train, y_train.ravel())
    lgb_eval = lgb.Dataset(x_valid, y_valid.ravel(), reference=lgb_train)

    # specify your configurations as a dict
    params = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': {'l2', 'auc'},
        'num_leaves': 31,
        'learning_rate': 0.05,
        'feature_fraction': 0.9,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'verbose': 0
    }

    print('Start training...')
    # train
    gbm = lgb.train(params,
                    lgb_train,
                    num_boost_round=20,
                    valid_sets=lgb_eval)

    preds = gbm.predict(x_valid)
    err = rmsle(np.exp(y_valid), preds)
    err_buf.append(err)
    print('RMSLE = ' + str(err))
    
    preds = model.predict(X_test)
    preds = np.exp(preds)
    preds_buf.append(preds)

print('Mean RMSLE = ' + str(np.mean(err_buf)) + ' +/- ' + str(np.std(err_buf)))

In [ ]:
from sklearn.datasets import load_iris
data = load_iris()
X = data.data
y = data.target

In [ ]:
X

In [ ]:
ROOT='/home/alexanderliao/data/Kaggle/competitions/tgs-salt-identification-challenge/'
train_path = os.path.join(ROOT, 'train')
train_file_list = glob.glob(os.path.join(train_path, 'images', '*.png'))
train_file_list = [f.split('/')[-1].split('.')[0] for f in train_file_list]
test_path = os.path.join(ROOT, 'test')
test_file_list = glob.glob(os.path.join(test_path, 'images', '*.png'))
test_file_list = [f.split('/')[-1].split('.')[0] for f in test_file_list]

In [ ]:
i=0
for file in train_file_list:
    if os.path.getsize(ROOT+'train/images/'+file+'.png')==107:
        i=i+1
print(i)

In [ ]:
i=0
for file in test_file_list:
    if os.path.getsize(ROOT+'test/images/'+file+'.png')==107:
        i=i+1
i

In [ ]:
1562/4000

In [ ]:
7614/18000-0.38

In [ ]:
(0.61-0.032)*0.855+0.38

In [ ]:
100-39